<a href="https://colab.research.google.com/github/canstralian/AI-Agent-Generating-Tool-Debugging-Prompt-Library/blob/main/inference_mamba_security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive',force_remount=True)
# !ln -s "/gdrive/My Drive/workspace" "/content/workspace"
# !ln -s "/gdrive/My Drive/home" "/content/home"
# !ln -s "/gdrive/My Drive/data" "/content/data"
# %env HOME=/content/drive/MyDrive/home
# !eval echo ~$USER

Mounted at /gdrive
ln: failed to create symbolic link '/content/workspace/workspace': File exists
ln: failed to create symbolic link '/content/home/home': File exists
ln: failed to create symbolic link '/content/data/data': File exists
env: HOME=/content/drive/MyDrive/home
/content/drive/MyDrive/home


In [ ]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



In [ ]:
!pip install -q causal-conv1d==1.0.0
!pip install -q mamba-ssm==1.0.1

In [ ]:
# from google.colab import output
# output.clear()
# del tokenizer
# del model

In [ ]:
import torch
from transformers import AutoTokenizer
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print out the device information
if device.type == 'cuda':
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("venkycs/mamba-2.8b-cybersec")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = MambaLMHeadModel.from_pretrained("venkycs/mamba-2.8b-cybersec")

# Move the model to the selected device
model = model.to(device)

# Rest of your code for model usage follows...


Using GPU: Tesla V100-SXM2-16GB


tokenizer_config.json:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.54G [00:00<?, ?B/s]

In [ ]:
model

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-63): 64 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)

In [ ]:
# Print memory footprint of the model
if device.type == 'cuda':
    print(f"Model memory footprint: {torch.cuda.memory_allocated(device)/1024**2:.2f} MB")
else:
    print("Memory footprint measurement is only available for GPU.")

Model memory footprint: 10561.38 MB


In [ ]:
model = model.half()

In [ ]:
# Print memory footprint of the model
if device.type == 'cuda':
    print(f"Model memory footprint: {torch.cuda.memory_allocated(device)/1024**2:.2f} MB")
else:
    print("Memory footprint measurement is only available for GPU.")

Model memory footprint: 5345.00 MB


In [ ]:
messages = []

# Informing the user about how to exit the chat
print("You can exit the chat anytime by typing 'bye' or 'quit'.")

while True:
    user_message = input("\nUser: ")

    # Check if the user's message is 'bye' or 'quit'
    if user_message.lower() in ['bye', 'quit']:
        print("Exiting the chat.")
        break

    messages.append(dict(
        role="user",
        content=user_message
    ))

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)

    out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(out)
    messages.append(dict(
        role="assistant",
        content=decoded[0].split("\n")[-2])
    )

    print("AI Assistant:", decoded[0].split("\n")[-1])


You can exit the chat anytime by typing 'bye' or 'quit'.

User: hi
AI Assistant: Hello! I'm a helpful AI Assistant and expert in Security Incident Handling. I help organizations develop Security Incident Handling related training with appropriate solutions. I provide specific scenarios and clear steps for better understanding.<|endoftext|>

User: How can AI be used to enhance network security?
AI Assistant: AI can be used to enhance network security in various ways, such as detecting and responding to threats in real-time, identifying anomalous behavior, and automating security incident response. AI can also help in predicting and preventing future security breaches by analyzing historical data and identifying patterns of attack.<|endoftext|>

User: Can you explain how the MITRE ATT&CK framework can be integrated into security training programs?
AI Assistant: Certainly! The MITRE ATT&CK framework can be integrated into security training programs by first familiarizing the participants 

In [ ]:
# Better to save locally, these model is not yet convered to hf
# tokenizer.save_pretrained('/content/home/mamba2.8b-cybersec')

('/content/home/mamba2.8b-cybersec/tokenizer_config.json',
 '/content/home/mamba2.8b-cybersec/special_tokens_map.json',
 '/content/home/mamba2.8b-cybersec/tokenizer.json')

In [ ]:
# save_directory = "/content/home/mamba2.8b-cybersec"
# # Save the model weights
# # model.save_pretrained('/content/home/mamba2.8b-cybersec')
# model_weights_path = os.path.join(save_directory, 'pytorch_model.bin')
# torch.save(model.state_dict(), model_weights_path)